In [13]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random
import os
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# 새로 업로드한 HTML 파일을 읽기
with open('Factiva.com.html', 'r', encoding='utf-8') as file:
    new_html_content = file.read()

# HTML 내용을 파싱
soup_new = BeautifulSoup(new_html_content, 'html.parser')

In [12]:
# # <p> 태그를 사용해 기사 내용을 찾아보기
# paragraphs = soup_new.find_all('p')

# # 기사 내용이 포함될 것으로 예상되는 긴 <p> 태그들을 찾기
# long_paragraphs = [p for p in paragraphs if len(p.get_text(strip=True)) > 50]

# # 확인을 위해 첫 번째로 발견된 긴 <p> 태그의 내용 출력
# long_paragraphs[0].get_text(strip=True) if long_paragraphs else "No long paragraphs found"


'Reuters News,   Wednesday,  06  September  2023,  01:19 GMT,  129 Words,  (English), Copyright 2023 Thomson Reuters. All Rights Reserved.(Document LBA0000020230905ej9503js1)'

In [3]:
paragraphs

NameError: name 'paragraphs' is not defined

In [4]:
# 각 <p> 태그의 내용을 분석하여 기사의 제목과 내용 추출
titles_new = []
contents_new = []

for p in long_paragraphs:
    text = p.get_text(strip=True)
    
    # 기사의 제목과 내용을 분리하기 위한 간단한 방법: 
    # 첫 번째 문장을 제목으로 간주하고 나머지를 본문으로 간주
    sentences = text.split('.')
    titles_new.append(sentences[0])
    contents_new.append(".".join(sentences[1:]))

# 결과를 DataFrame 형식으로 변환
df_new = pd.DataFrame({
    'Title': titles_new,
    'Content': contents_new
})

# DataFrame의 첫 5개 행을 출력하여 확인
df_new.head()


NameError: name 'long_paragraphs' is not defined

In [5]:
# Read the new text file content
with open("Factiva-07September20231123.txt", "r", encoding="ISO-8859-1") as file:
    new_txt_content = file.read()

In [6]:
# Final code to split the articles based on the "Document XXXX..." pattern, keeping the delimiter at the end,
# and then remove the "Document XXXX..." pattern from the start of each article

# Split the content based on the "Document XXXX..." pattern, keeping the delimiter at the end
articles_with_document_split = re.split(r'(Document \w{8}\d{8}\w{7})', new_txt_content)

# Adjust the split to form articles with the "Document XXXX..." pattern only at the end
adjusted_articles_final = [articles_with_document_split[i+1] + articles_with_document_split[i] 
                          for i in range(0, len(articles_with_document_split)-1, 2)]

# Convert the adjusted articles into a DataFrame
df_adjusted_articles_final = pd.DataFrame(adjusted_articles_final, columns=["Article_Content"])

# Remove the "Document XXXX..." pattern from the start of each article
df_adjusted_articles_final["Article_Content"] = df_adjusted_articles_final["Article_Content"].str.replace(r'^Document \w{8}\d{8}\w{7}', '', regex=True)

df_adjusted_articles_final["Article_Content"] = df_adjusted_articles_final["Article_Content"].str.replace(r'^.*?\n+', '', regex=True)


In [12]:
df_adjusted_articles_final['Article_Content'][0]

'Business\nDigital currencies: CBA to limit fund transfers in crypto scam crackdown\n\nDominic Powell \n485 words\n9 June 2023\nThe Sydney Morning Herald\nSMHH\nFirst\n26\nEnglish\n? 2023 Copyright John Fairfax Holdings Limited. www.smh.com.au\n\nCommonwealth Bank will limit customers from sending money to certain cryptocurrency exchanges and implement monthly limits on crypto purchases in an effort to crack down on scams and fraud.\n\nYesterday, the big four bank introduced a raft of new scam prevention methods, including bans or 24-hour holds on payments to certain crypto exchanges, becoming the second major bank to do so after Westpac took similar action last month.\n\nCBA would not identify the exchanges that would be limited under the new system. However, it\'s likely Binance - one of the world\'s largest and most popular cryptocurrency exchanges - will be included.\n\nThis week, US regulators took action against Binance and Coinbase, another major exchange, accusing them of viola

In [12]:
# Extract title, content, and publication date from each article

# Extract title (assuming title is the first line of the article)
df_adjusted_articles_final['Title'] = df_adjusted_articles_final['Article_Content'].str.split('\n').str[0]

# Extract content (assuming content is everything after the title and before the last line which contains the Document pattern)
df_adjusted_articles_final['Content'] = df_adjusted_articles_final['Article_Content'].apply(lambda x: '\n'.join(x.split('\n')[1:-1]))

# Extract publication date from the "Document XXXX..." pattern
df_adjusted_articles_final['Publication_Date'] = df_adjusted_articles_final['Article_Content'].str.extract(r'Document \w{4}(\d{8})\w{7}')

# Drop the original combined column
df_extracted_data = df_adjusted_articles_final.drop(columns=["Article_Content"])

# Display the first few rows of the extracted DataFrame
df_extracted_data.head()


,Title,Content,Publication_Date
0,Business,Digital currencies: CBA to limit fund transfer...,NaN
1,General News,Winter is here for exchanges\n\nLex comment \n...,NaN
2,Business,CBA s crypto block to hit scammers\n\n144 wor...,NaN
3,Business,"Greatest innovation ever: AI to slash prices, ...",NaN
4,News,Banks put the brake on crypto\n\nBen Butler \n...,NaN


In [32]:
# 웹 드라이버 초기화 (예: Chrome을 사용)
driver = webdriver.Chrome(executable_path=r'C:\Users\user\chromedriver\chromedriver.exe')

# 대상 웹사이트로 이동
driver.get("https://snapshot.factiva.com/Search/SSResults")

# 30번 반복
for i in range(30):
    # .article-view-trigger 클래스를 가진 a태그를 찾음
    articles = driver.find_elements(By.CSS_SELECTOR, ".article-view-trigger")
    
    # 첫 번째 항목 클릭
    articles[0].click()
    
    # 페이지 로딩을 기다림 (필요에 따라 시간 조정 가능)
    time.sleep(5)
    
    # 뒤로 가기
    driver.back()
    
    # 페이지 로딩을 기다림 (필요에 따라 시간 조정 가능)
    time.sleep(5)

# 작업 완료 후 웹 드라이버 종료
driver.quit()


TypeError: __init__() got an unexpected keyword argument 'executable_path'